In [1]:
# automatically reload dependant notebooks
%load_ext autoreload
%autoreload 2
import import_ipynb

# All-Pairs Shortest Paths

Given a directed, weighted graph $G = (V, E)$, All-Pairs Shortest Path (ASP) algorithm discovers the shortest paths between each pair of vertices. See CLRS 4ed Chapter 23 *All-Pairs Shortest Paths* p.646. ASP can be used to tabulate the cheapest airfares between pairs of city. The ticket price from city $C$ to $D$ is not necessarily the same as that from $D$ to $C$, due to day and time of travel, airlines in operation, flight availability, stopovers, etc.

## ASP graph with adjacency matrix of edge weights

Unlike those algorithms in the earlier chapters, ASP algorithms use dense graphs represented with adjacency matrices. So, we derive `ASPGraph` from `MtxGraph`. See Equation 23.1 p.647 for the description of adjacency matrix representation. Note that we use `WgtEdge` defined in the [mst.ipynb](./mst.ipynb) notebook.

In [2]:
from graph import *
from mst import *
from asp import *
from util import *

class ASPGraph(MtxGraph):
  def __init__(self, tag: Tag):
    super().__init__(tag)

  def makeVEw(self, vt: [Tag], et: [Tag], ew: {Tag, float}) -> None:
    self.makeV(vt)
    self.makeEw(et, ew)
  def makeEw(self, et: [Tag], ew: {Tag, float}) -> None:
    for etag in et:
      [utag, vtag] = parseETag(etag)
      e = WgtEdge(self.getV(utag), self.getV(vtag), float(ew[etag]))
      self.ee[e.tag] = e
      [i, j] = indicesOfETag(etag)
      self.ww[i][j] = ew[etag]

importing Jupyter notebook from graph.ipynb
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
importing Jupyter notebook from util.ipynb
importing Jupyter notebook from mst.ipynb
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
importing Jupyter notebook from asp.ipynb
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
importing Jupyter notebook from ssp.ipynb
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
importing Jupyter notebook from ega.ipynb
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Floyd-Warshall ASP algorithm

Floyd-Warshall ASP algorithm employs dynamic programming to find the optimal (shortest) paths between every pair of vertices. See 23.2 The Floyd-Warshall algorithm p.655. The runtime of the Floyd-Warshall algorithm is $\Theta(V^3)$. See p.657.

[Dynamic programming](https://en.wikipedia.org/wiki/Dynamic_programming) is a tabulation-based optimisation technique. [Greedy algorithms](https://en.wikipedia.org/wiki/Greedy_algorithm), like Kruskal's MST, Prim's MST, and Dijkstra's SSP, select at each optimisation step a locally optimal choice. But dynamic programming algorithms select at each optimisation step the globally optimal choice. See Chapter 14 Dynamic Programming p.362.

Do not confuse dynamic programming with [dynamic programming languages](https://en.wikipedia.org/wiki/Dynamic_programming_language)—like Python.

In [3]:
def aspFloydWarshall(g: ASPGraph) -> [WMtx, WMtx]:
  n = g.numVV()
  r = range(0, n)
  np1 = n + 1
  # initialize
  dd = [[]] * np1
  dd[0] = g.ww
  pp = [[]] * np1
  pp[0] = [[]] * n
  for i in r:
    pp[0][i] = [-Infinity] * n  # use -Infinity instead of NIL as used in CLRS; see p.659
    for j in r: pp[0][i][j] = -Infinity if i == j or g.ww[i][j] == Infinity else i
  # discover ASP in graph g
  for k in range(1, np1):
    km1 = k - 1
    dd[k] = [[]] * n
    pp[k] = [[]] * n
    for i in r:
      dd[k][i] = [Infinity] * n
      pp[k][i] = [-Infinity] * n
      for j in r:
        dd[k][i][j] = min(dd[km1][i][j], dd[km1][i][km1] + dd[km1][km1][j])
        pp[k][i][j] = pp[km1][km1][j] if dd[km1][i][j] > dd[km1][i][km1] + dd[km1][km1][j] else pp[km1][i][j]  # see Equation 23.8 p.659
  return dd[n], pp[n]

## transitive closure of a directed graph

The transitive closure of a directed graph $G = (V, E)$ is $G^* = (V, E^*)$, where $E^* = \{(i, j)\, |\, i \leadsto j \in G\}$. See p.659. The transitive closure algorithm on p.660 uses the `tt` boolean matrix, so we define `BMtx`, below.

In [4]:
BMtx = [[bool]]

def tclosure(g: ASPGraph) -> BMtx:
  n = g.numVV()
  r = range(0, n)
  np1 = n + 1
  # intialize
  tt: BMtx = [[]] * np1
  tt[0] = [[]] * n
  for i in r:
    tt[0][i] = [[]] * n
    for j in r: tt[0][i][j] = i == j or g.hasE(etagOfIndices(i, j))
  for k in range(1, np1):
    km1 = k - 1
    tt[k] = [[]] * n
    for i in r:
      tt[k][i] = [False] * n
      for j in r: tt[k][i][j] = tt[km1][i][j] or (tt[km1][i][km1] and tt[km1][km1][j])
  return tt[n]

## Johnson's ASP algorithm

Johnson's ASP algorithm is appreciably faster than Floyd-Warshall. Instead of using triple-nested loops like Floyd-Warshall, Johnson's algorithm applies Dijkstra's SSP algorithm, once for each vertex as the source. But since negative edge weights are allowed, Dijkstra's algorithm cannot be applied directly. So, this algorithm uses a clever trick, called *reweighting*, to make all weights positive, without altering the shortest path structures of the graph. See 23.3 *Johnson’s algorithm for sparse graphs* p.662. The runtime of Johnson's algorithm is $O(V^2\, lg\, V + VE)$. See p.662.

In [5]:
from copy import deepcopy
from ssp import *

JohnsonGraph = DijkstraGraph  # uses PriVert and WgtEdge

def aspJohnson(g: JohnsonGraph) -> [WMtx, [LstTree]]:
  # initialize
  h: JohnsonGraph = JohnsonGraph(f"{g.tag}+")  # graph h is graph g augmented with source vertex s and its out edges
  h.dupVV(g.vv)
  h.dupEE(g.ee)
  s = Vert("0")
  h.insV(s)
  for u in g.getVV():
    e = WgtEdge(s, u)
    e.wgt = 0.0
    h.insE(e)
  # check for negative-weight cycles using Bellman-Ford
  p = sspBellmanFord(h, s)
  if isNone(p): raise Exception("input graph contains a negative-weight cycle")
  # reweight graph h to eliminate negative weights on edges
  for e in h.getEE():  # for each edge in graph h
    u = h.getV(e.u.tag)
    v = h.getV(e.v.tag)
    # w^(u, v) = w(u, v) + h(u) - h(v), where h(u) = δ(s, u) and h(v) = δ(s, v); see Equation 23.10 p.663 and p.664
    e.wgt += u.dis - v.dis
  # discover ASP in graph g using Dijkstra, once for each vertex as the source
  n = g.numVV()
  r = range(0, n)
  dd = [[]] * n  # shortest distances
  tt = [LstTree] * n  # Dijkstra SSPs
  for i in r: dd[i] = [float] * n
  g.tag = f"{g.tag}ω"
  g.dupVV(h.vv)  # copy u.dis = δ(s, u)
  g.delV(s)
  for u in g.getVV():  # for each vertex in graph g
    i = int(u.tag) - 1
    # discover SSP in graph g from vertex u
    p = sspDijkstra(g, u)  # compute δ^(u, v) for all vertices v of graph g
    tt[i] = deepcopy(p)
    for v in g.getVV():
      j = int(v.tag) - 1
      dd[i][j] = u.pri + v.dis - u.dis
  return dd, tt

# Conclusion

In this notebook, we implemented Floyd-Warshall ASP algorithm for dense graphs, transitive closure of a directed graph, and Johnson's ASP algorithm for sparse graphs. Tests for these ASP algorithms are implemented in [`asptest.ipynb`](./asptest.ipynb).